#Import libraries


In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

from rdkit.Chem import PandasTools, Descriptors, rdmolops
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.rdmolfiles import MolFromSmiles

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#Import database

In [ ]:
df = pd.read_excel('Merge_data.xlsx')
df.head()

,Index,Authors,ID,Name,SMILES,MW,Subclass,Origin,Glucosidase,Acarbose,...,Test.1,Mechanism.1,Acarbose.2,Test.2,Ratio,Acarbose.3,Test.3,Ratio.1,Note,Both enzyme
0,1,Yang Yao et al. (2011),1,Vitexin,O=C1C=C(C2=CC=C(O)C=C2)OC3=C1C(O)=CC(O)=C3[C@@...,NaN,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
1,1,Yang Yao et al. (2011),2,Isovitexin,O=C1C=C(C2=CC=C(O)C=C2)OC3=C1C(O)=C([C@@H]4O[C...,NaN,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
2,2,"Van Thanh, B. et al. (2022)",2,Dysosmaflavanone,O=C1[C@](CCC(C)=O)(O)[C@](C2=CC=C(O)C=C2)(OCC)...,NaN,Flavanonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1382,NaN,623.9,1330,NaN,NaN,yes
3,2,"Van Thanh, B. et al. (2022)",3,Podoverine A,O=C1C(OC)=C(C2=CC=C(O)C(O)=C2C/C=C(C)\C)OC3=C1...,NaN,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1962,NaN,623.9,1297,NaN,NaN,yes
4,2,"Van Thanh, B. et al. (2022)",4,Kaempferol,OC1=C(C2=CC=C(O)C=C2)OC3=C(C(O)=CC(O)=C3)C1=O,NaN,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1360,NaN,623.9,1246,NaN,NaN,yes


#Standardize SMILE strings and calculate molecular weight (MW)

In [ ]:
smis = df["SMILES"]
uncharger = rdMolStandardize.Uncharger()
ms=[]
for smi in smis:
    m = MolFromSmiles(smi)
    # Sanitize
    Chem.SanitizeMol(m, sanitizeOps = (Chem.SANITIZE_ALL^Chem.SANITIZE_CLEANUP^Chem.SANITIZE_PROPERTIES))
    # Normalize functional groups
    cm = rdMolStandardize.Normalize(m)
    # Append to list
    ms.append(Chem.MolToSmiles(cm))
#Draw.MolsToGridImage(ms,molsPerRow=2,legends=['before','uncharged parent']*(len(ms)//2), subImgSize=(350,300))
df["SMILES_standardized"]=ms
df['MW'] = [Descriptors.MolWt(Chem.MolFromSmiles(x)) for x in df.SMILES_standardized]
df.drop('SMILES',axis=1,inplace=True)

[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing Normalizer
[03:23:44] Running Normalizer
[03:23:44] Initializing N

In [ ]:
df=df[['Index', 'Authors', 'ID', 'Name', 'SMILES_standardized','MW', 'Subclass', 'Origin',
       'Glucosidase ', 'Acarbose', 'Test', 'Mechanism', 'Amylase',
       'Acarbose.1', 'Test.1', 'Mechanism.1', 'Acarbose.2', 'Test.2', 'Ratio',
       'Acarbose.3', 'Test.3', 'Ratio.1', 'Note', 'Both enzyme'
       ]]
df.head()

,Index,Authors,ID,Name,SMILES_standardized,MW,Subclass,Origin,Glucosidase,Acarbose,...,Test.1,Mechanism.1,Acarbose.2,Test.2,Ratio,Acarbose.3,Test.3,Ratio.1,Note,Both enzyme
0,1,Yang Yao et al. (2011),1,Vitexin,O=c1cc(-c2ccc(O)cc2)oc2c([C@@H]3O[C@H](CO)[C@@...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
1,1,Yang Yao et al. (2011),2,Isovitexin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)c([C@@H]3O[C@H](CO...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
2,2,"Van Thanh, B. et al. (2022)",2,Dysosmaflavanone,CCO[C@]1(c2ccc(O)cc2)Oc2cc(O)cc(O)c2C(=O)[C@@]...,402.399,Flavanonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1382,NaN,623.9,1330,NaN,NaN,yes
3,2,"Van Thanh, B. et al. (2022)",3,Podoverine A,COc1c(-c2ccc(O)c(O)c2CC=C(C)C)oc2cc(O)cc(O)c2c1=O,384.384,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1962,NaN,623.9,1297,NaN,NaN,yes
4,2,"Van Thanh, B. et al. (2022)",4,Kaempferol,O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,286.239,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1360,NaN,623.9,1246,NaN,NaN,yes




```
#Comment for columns:
#Acarbose: IC50 of acarbose against glucosidase in mg/mL
#Test: IC50 of flavonoid against glucosidase in mg/ml
#Acarbose.1: IC50 of acarbose against amylase in mg/mL
#Test.1: IC50 of flavonoid against amylase in mg/ml
#Acarbose.2: IC50 of acarbose against glucosidase in uM
#Test.2: IC50 of flavonoid against glucosidase in uM
#Acarbose.3: IC50 of acarbose against amylase in uM
#Test.3: IC50 of flavonoid against amylase in uM
```



In [ ]:
dtype_cols = ['Acarbose', 'Test', 'Acarbose.1', 'Test.1',
              'Acarbose.2', 'Test.2', 'Acarbose.3', 'Test.3']

In [ ]:
for col in dtype_cols:
  df[col] = df[col].fillna('-1')
  df[col] = df[col].astype('string')
  df[col] = df[col].apply(lambda x : x.split('Â±')[0])
  df[col].str.strip()

In [ ]:
for col in dtype_cols:
  df[col] = df[col].replace('-1', np.nan)
  df[col] = df[col].replace('nd', np.nan)
  df[col] = df[col].replace('ND', np.nan)
  df[col] = df[col].replace('x', np.nan)

In [ ]:
mg_cols = ['Acarbose', 'Test', 'Acarbose.1', 'Test.1']

for col in mg_cols:
  df[col] = pd.to_numeric(df[col], errors='raise')

In [ ]:
uM = ['Acarbose.2', 'Test.2', 'Acarbose.3', 'Test.3']
mg_cols = ['Acarbose', 'Test', 'Acarbose.1', 'Test.1']

#Calculate MW and IC50 in molarity for acarbose

In [ ]:
print(Descriptors.MolWt(Chem.MolFromSmiles('CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OC3C(OC(C(C3O)O)O)CO)CO)O)O)NC4C=C(C(C(C4O)O)O)CO')))

645.6080000000004


In [ ]:
mg_cols = ['Acarbose', 'Test', 'Acarbose.1', 'Test.1']
uM =      ['Acarbose.2', 'Test.2', 'Acarbose.3', 'Test.3']

In [ ]:
#Calculate IC50 for acarbose in molarity from the studies which represented IC50 in mg/ml
select = (df['Acarbose.2'].isna()) & (df['Acarbose'].notna())
df_selected = df[select]
df.loc[select, 'Acarbose.2'] = df_selected['Acarbose'] / 645.608 * 1e06

select = (df['Acarbose.3'].isna()) & (df['Acarbose.1'].notna())
df_selected = df[select]
df.loc[select, 'Acarbose.3'] = df_selected['Acarbose.1'] / 645.608 * 1e06

#Caculate IC50 in molarity for flavonoids

In [ ]:
select = (df['Test.2'].isna()) & (df['Test'].notna())
df_selected = df[select]

df.loc[select, 'Test.2'] = df_selected['Test'] / df_selected['MW'] * 1e06

In [ ]:
select = (df['Test.3'].isna()) & (df['Test.1'].notna())
df_selected = df[select]

df.loc[select, 'Test.3'] = df_selected['Test.1'] / df_selected['MW'] * 1e06

# Review and output

In [ ]:
df.head()

,Index,Authors,ID,Name,SMILES_standardized,MW,Subclass,Origin,Glucosidase,Acarbose,...,Test.1,Mechanism.1,Acarbose.2,Test.2,Ratio,Acarbose.3,Test.3,Ratio.1,Note,Both enzyme
0,1,Yang Yao et al. (2011),1,Vitexin,O=c1cc(-c2ccc(O)cc2)oc2c([C@@H]3O[C@H](CO)[C@@...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,925.11003,NaN,NaN,NaN,NaN,NaN,no
1,1,Yang Yao et al. (2011),2,Isovitexin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)c([C@@H]3O[C@H](CO...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,11101.320363,NaN,NaN,NaN,NaN,NaN,no
2,2,"Van Thanh, B. et al. (2022)",2,Dysosmaflavanone,CCO[C@]1(c2ccc(O)cc2)Oc2cc(O)cc(O)c2C(=O)[C@@]...,402.399,Flavanonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1382,NaN,623.9,1330,NaN,NaN,yes
3,2,"Van Thanh, B. et al. (2022)",3,Podoverine A,COc1c(-c2ccc(O)c(O)c2CC=C(C)C)oc2cc(O)cc(O)c2c1=O,384.384,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1962,NaN,623.9,1297,NaN,NaN,yes
4,2,"Van Thanh, B. et al. (2022)",4,Kaempferol,O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,286.239,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020,1360,NaN,623.9,1246,NaN,NaN,yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1624 entries, 0 to 1623
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Index                1624 non-null   int64  
 1   Authors              1624 non-null   object 
 2   ID                   1496 non-null   object 
 3   Name                 1582 non-null   object 
 4   SMILES_standardized  1624 non-null   object 
 5   MW                   1624 non-null   float64
 6   Subclass             1622 non-null   object 
 7   Origin               1623 non-null   object 
 8   Glucosidase          1471 non-null   object 
 9   Acarbose             268 non-null    float64
 10  Test                 265 non-null    float64
 11  Mechanism            637 non-null    object 
 12  Amylase              372 non-null    object 
 13  Acarbose.1           102 non-null    float64
 14  Test.1               100 non-null    float64
 15  Mechanism.1          131 non-null    o

#Recheck

In [ ]:
df.to_csv('Merge_data_out.csv', index = None)
df = pd.read_csv('Merge_data_out.csv')
uM_cols = ['Acarbose.2', 'Test.2', 'Acarbose.3', 'Test.3']

for col in uM_cols:
  df[col] = pd.to_numeric(df[col], errors = 'ignore')

In [ ]:
#Calculate ratio based on the value of IC50 of acarbose and flavonoids

In [ ]:
#Check if any ratio cell has not been calculated
select = (df['Ratio'].isna()) & (df['Acarbose.2'].notna()) & (df['Test.2'].notna())
df_selected = df[select]

df_selected['Test.2'] = pd.to_numeric(df_selected['Test.2'], errors = 'raise')
df_selected['Acarbose.2'] = pd.to_numeric(df_selected['Acarbose.2'], errors = 'raise')

df.loc[select, 'Ratio'] = np.around(df_selected['Test.2'] / df_selected['Acarbose.2'], decimals = 2)

<ipython-input-21-b50246baee55>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Test.2'] = pd.to_numeric(df_selected['Test.2'], errors = 'raise')
<ipython-input-21-b50246baee55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Acarbose.2'] = pd.to_numeric(df_selected['Acarbose.2'], errors = 'raise')


In [ ]:
select = (df['Ratio.1'].isna()) & (df['Acarbose.3'].notna()) & (df['Test.3'].notna())
df_selected = df[select]

df_selected['Test.3'] = pd.to_numeric(df_selected['Test.3'], errors = 'raise')
df_selected['Acarbose.3'] = pd.to_numeric(df_selected['Acarbose.3'], errors = 'raise')

df.loc[select, 'Ratio.1'] = np.around(df_selected['Test.3'] / df_selected['Acarbose.3'], decimals = 2)

<ipython-input-22-f9ba568e8a1d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Test.3'] = pd.to_numeric(df_selected['Test.3'], errors = 'raise')
<ipython-input-22-f9ba568e8a1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Acarbose.3'] = pd.to_numeric(df_selected['Acarbose.3'], errors = 'raise')


In [ ]:
df.head()

,Index,Authors,ID,Name,SMILES_standardized,MW,Subclass,Origin,Glucosidase,Acarbose,...,Test.1,Mechanism.1,Acarbose.2,Test.2,Ratio,Acarbose.3,Test.3,Ratio.1,Note,Both enzyme
0,1,Yang Yao et al. (2011),1,Vitexin,O=c1cc(-c2ccc(O)cc2)oc2c([C@@H]3O[C@H](CO)[C@@...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,925.110030,1.33,NaN,NaN,NaN,NaN,no
1,1,Yang Yao et al. (2011),2,Isovitexin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)c([C@@H]3O[C@H](CO...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,11101.320363,15.93,NaN,NaN,NaN,NaN,no
2,2,"Van Thanh, B. et al. (2022)",2,Dysosmaflavanone,CCO[C@]1(c2ccc(O)cc2)Oc2cc(O)cc(O)c2C(=O)[C@@]...,402.399,Flavanonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1382.000000,1.35,623.9,1330.0,2.13,NaN,yes
3,2,"Van Thanh, B. et al. (2022)",3,Podoverine A,COc1c(-c2ccc(O)c(O)c2CC=C(C)C)oc2cc(O)cc(O)c2c1=O,384.384,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1962.000000,1.92,623.9,1297.0,2.08,NaN,yes
4,2,"Van Thanh, B. et al. (2022)",4,Kaempferol,O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,286.239,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1360.000000,1.33,623.9,1246.0,2.00,NaN,yes


In [ ]:
df['Both enzyme'] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1624 entries, 0 to 1623
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Index                1624 non-null   int64  
 1   Authors              1624 non-null   object 
 2   ID                   1496 non-null   object 
 3   Name                 1582 non-null   object 
 4   SMILES_standardized  1624 non-null   object 
 5   MW                   1624 non-null   float64
 6   Subclass             1622 non-null   object 
 7   Origin               1623 non-null   object 
 8   Glucosidase          1471 non-null   object 
 9   Acarbose             268 non-null    float64
 10  Test                 265 non-null    float64
 11  Mechanism            637 non-null    object 
 12  Amylase              372 non-null    object 
 13  Acarbose.1           102 non-null    float64
 14  Test.1               100 non-null    float64
 15  Mechanism.1          131 non-null    o

In [ ]:
select = (df['Ratio'].notna()) & (df['Ratio.1'].notna())
df.loc[select, 'Both enzyme'] = 'yes'

In [ ]:
df['Both enzyme'] = df['Both enzyme'].fillna(value = 'no')
df.head()

,Index,Authors,ID,Name,SMILES_standardized,MW,Subclass,Origin,Glucosidase,Acarbose,...,Test.1,Mechanism.1,Acarbose.2,Test.2,Ratio,Acarbose.3,Test.3,Ratio.1,Note,Both enzyme
0,1,Yang Yao et al. (2011),1,Vitexin,O=c1cc(-c2ccc(O)cc2)oc2c([C@@H]3O[C@H](CO)[C@@...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,925.110030,1.33,NaN,NaN,NaN,NaN,no
1,1,Yang Yao et al. (2011),2,Isovitexin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)c([C@@H]3O[C@H](CO...,432.381,Flavone,Vigna angularis,Chromogenic,0.45,...,NaN,NaN,697.017385,11101.320363,15.93,NaN,NaN,NaN,NaN,no
2,2,"Van Thanh, B. et al. (2022)",2,Dysosmaflavanone,CCO[C@]1(c2ccc(O)cc2)Oc2cc(O)cc(O)c2C(=O)[C@@]...,402.399,Flavanonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1382.000000,1.35,623.9,1330.0,2.13,NaN,yes
3,2,"Van Thanh, B. et al. (2022)",3,Podoverine A,COc1c(-c2ccc(O)c(O)c2CC=C(C)C)oc2cc(O)cc(O)c2c1=O,384.384,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1962.000000,1.92,623.9,1297.0,2.08,NaN,yes
4,2,"Van Thanh, B. et al. (2022)",4,Kaempferol,O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,286.239,Flavonol,Dysosma difformis,Chromogenic,NaN,...,NaN,NaN,1020.000000,1360.000000,1.33,623.9,1246.0,2.00,NaN,yes


In [ ]:
df['Both enzyme'].value_counts()

no     1422
yes     202
Name: Both enzyme, dtype: int64

In [ ]:
df.to_csv('Merge_data_out.csv', index = None)